In [2]:
import pandas as pd
import numpy as np

---

# Análisis para encontrar futuros targets

In [3]:
events = pd.read_pickle("../../../../data/tp2/events_tp2_formateado.pkl")

In [4]:
events.drop(columns = 'index',inplace = True)

### ¿Cúales son los eventos más frecuentes? 
### ¿Hay algún top de eventos que compartan todos los dispositivos al menos una vez en cada ventana?

In [5]:
grupo = events.groupby(["ref_hash","event_id"]).size().to_frame()
idx = grupo.groupby(['ref_hash'])[0].transform(max) == grupo[0]
grupo = grupo[idx]

In [7]:
grupo.rename(columns= {0:"cant_ocurrencias"},inplace = True)

In [8]:
grupo.head()

,,cant_ocurrencias
ref_hash,event_id,
40621409780134,363,7
41863526108385,1,100
69039685746313,287,12
90072729247980,0,3
161514654074162,15,31


#### ¿Hay dispositivos que tengan más de un evento como máximo ?

In [9]:
((grupo.groupby(["ref_hash"]).count()["cant_ocurrencias"]) > 1).sum()

53860

#### ¿Cual es el máximo numero de eventos empatados como máximo por dispositivo?

In [10]:
(grupo.groupby(["ref_hash"]).count()).max()

cant_ocurrencias    10
dtype: int64

#### ¿Podría usarse algún criterio se toma para decidir con cual evento quedarse dentro de los dispositivos que tienen más de un evento como máximo? 

##### Voy a crear una lista ordenada de los eventos más presentes por dispositivo, contando un solo dispositivo por evento para no desbalancear si un dispositivo hace 10000 veces el mismo evento. Esto podría dar un orden de importancia entre eventos

In [12]:
top_eventos = grupo.groupby(["ref_hash","event_id"]).size().reset_index()\
                      .groupby("event_id").size().sort_values(ascending = False)

In [13]:
top_eventos.head()

event_id
23    105181
15     75063
7      36674
1      24686
0      20867
dtype: int64

top_eventos = top_eventos.index.to_list()

##### Esta lista se podría usar para tener los eventos ordenados por importancia y si un dispositivo tiene más de un evento, quedarnos con el que pirmero aparezca dentro de la lista.

##### Propuesta: También se podrían usar otros criterios para definir importancia de eventos, por ejemplo filtrar entre eventos antes de una subasta o una conversión que es lo que nos importa

### ¿Podría usarse la aplicación más usada como feature ? ¿Quizás poniendo una etiqueta que nos diga el horario de uso de esa aplicación o qué día es más frecuente que la use?

### ¿Qué día de la semana es más frecuente un usuario? Quizás se puede mapear el día de la semana con un puntaje, dando menor peso a los días donde no apareció ninguna vez y más a los que apareció con más frecuencia, esto podría ayudarnos con ventanas en las que quizás no tenemos muchos datos de una persona pero sabemos que su día más frecuente esta dentro de los próximos por lo que tendría oportunidad de aparecer